<a href="https://colab.research.google.com/github/Arif432/portfolios/blob/main/bertModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Load the model via the transformers library
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")


In [3]:
clinical_note = "Patient presents with chest pain and shortness of breath."
tokenized_input = tokenizer.encode_plus(clinical_note, return_tensors="pt", padding=True, truncation=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [7]:
import torch

with torch.no_grad():
    output = model(**tokenized_input)
    embeddings = output.last_hidden_state


In [8]:
print(embeddings)


tensor([[[ 0.0894,  0.2792, -0.2372,  ..., -0.5050,  0.0806, -0.3876],
         [-0.2173, -0.1029,  0.1795,  ...,  0.0390, -0.0651,  0.0905],
         [ 0.1883,  0.1502,  0.4540,  ...,  0.1421, -0.2161, -0.1014],
         ...,
         [ 0.2611,  0.0791,  0.1329,  ..., -0.4998, -0.1079, -0.4049],
         [ 0.6670, -0.2609, -1.0834,  ..., -0.2224,  0.1810, -0.6438],
         [ 0.7426, -0.3093, -1.0478,  ..., -0.2260,  0.1855, -0.6380]]])


In [9]:
# Define a function to process user input
def process_user_input(user_query):
    # Tokenize the input
    tokenized_input = tokenizer.encode_plus(user_query, return_tensors="pt", padding=True, truncation=True)

    # Get contextualized embeddings
    with torch.no_grad():
        output = model(**tokenized_input)
        embeddings = output.last_hidden_state

    symptom_embedding = embeddings[0]
    symptom = tokenizer.decode(tokenized_input["input_ids"][0])

    return symptom

# Example usage
user_query = "Hi can you please help me with diabetes?"
symptom = process_user_input(user_query)
print(f"Detected symptom: {symptom}")


Detected symptom: [CLS] hi can you please help me with diabetes? [SEP]


In [14]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Load pre-trained Clinical BERT model and tokenizer
model = BertForQuestionAnswering.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
tokenizer = BertTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# Input text
input_text = "What are the symptoms of diabetes?"

# Tokenize input
inputs = tokenizer(input_text, return_tensors="pt")

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Decode the output
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits)
answer = tokenizer.decode(inputs.input_ids[0][answer_start:answer_end+1])

print("Answer:", answer)



Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Answer: the symptoms
